## Import Packages

In [1]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import time
import os
from scipy.ndimage import gaussian_filter1d
from nlb_tools.make_tensors import h5_to_dict, save_to_h5
from nlb_tools.evaluation import evaluate
import h5py

## GPU Setting

In [2]:
tStart = time.time()
print(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

cuda


## Sub Functions

In [3]:
def _GF(ddd, sig, N, C, log=True):
    TL = []
    for n in range(N):
        CL = []
        spike = ddd[n, :, :]
        for c in range(C):
            sp_c = spike[:, c]
            sp_c_gf = gaussian_filter1d(sp_c.astype(np.float32), sig)
            if log:
                CL.append(np.log(sp_c_gf + 1e-10)[:, np.newaxis])
            else:
                CL.append(sp_c_gf[:, np.newaxis])
        TL.append(np.hstack(CL)[np.newaxis, :, :])
    TL = np.vstack(TL)
    return TL

def _Dict(dataset_name, train, valid, T, C):
    train_rates_heldin = train[:, :T, :C]
    train_rates_heldout = train[:, :T, C:]
    eval_rates_heldin = valid[:, :T, :C]
    eval_rates_heldout = valid[:, :T, C:]
    eval_rates_heldin_forward = valid[:, T:, :C]
    eval_rates_heldout_forward = valid[:, T:, C:]

    output_dict = {
        dataset_name: {
            'train_rates_heldin': train_rates_heldin.astype(np.float64),
            'train_rates_heldout': train_rates_heldout.astype(np.float64),
            'eval_rates_heldin': eval_rates_heldin.astype(np.float64),
            'eval_rates_heldout': eval_rates_heldout.astype(np.float64),
            'eval_rates_heldin_forward': eval_rates_heldin_forward.astype(np.float64),
            'eval_rates_heldout_forward': eval_rates_heldout_forward.astype(np.float64)
        }
    }  
    return output_dict

def _shuffle(spk_hi):
    spk_hi_shf = spk_hi[:, :, torch.randperm(spk_hi.size()[2])]
    return spk_hi_shf

def _GF_pt(spk_pt, sig, N, C, log=True):
    spk_np = spk_pt.data.numpy()
    spk_GF_np = _GF(spk_np, sig, N, C, log)
    return torch.from_numpy(spk_GF_np).type(torch.FloatTensor)


def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


## Create Dictionary

In [4]:
dataset_dict = {
    '000128': 'mc_maze',
    '000127': 'area2_bump',
    '000130': 'dmfc_rsg',
    '000129': 'mc_rtt',
    '000138': 'mc_maze_large'
}
hT_dict = {
    '000128': 7,
    '000127': 6,
    '000130': 15,
    '000129': 6,
    '000138': 7
}
sP = './inFR'
dpath = './data/h5'
data_list = os.listdir('./data/h5')

# for idx in ['000127', '000128', '000129', '000130', '000138']:
idx = '000128'
dataset_name = dataset_dict[idx]
print('\n==========' + dataset_name + '==========')
for d in data_list:
    if d.split('_')[0]==idx:
        if d.split('_')[1]=='train':
            train_F = d
        elif d.split('_')[1]=='eval':
            valid_F = d
        elif d.split('_')[1]=='test':
            test_F = d   
        elif d.split('_')[1]=='target':
            target_F = d               




==========mc_maze==========


## Data Processing

In [5]:
train_H5 = h5py.File(os.path.join(dpath, train_F), 'r')
valid_H5 = h5py.File(os.path.join(dpath, valid_F), 'r')
test_H5 = h5py.File(os.path.join(dpath, test_F), 'r')
target_dict = h5_to_dict(h5py.File(os.path.join(dpath, target_F), 'r'))

train_spikes_heldin  = np.array(train_H5['train_spikes_heldin'])
train_spikes_heldin_forward  = np.array(train_H5['train_spikes_heldin_forward'])
train_spikes_heldout  = np.array(train_H5['train_spikes_heldout'])
train_spikes_heldout_forward  = np.array(train_H5['train_spikes_heldout_forward'])

eval_spikes_heldin = np.array(valid_H5['eval_spikes_heldin'])
eval_spikes_heldout = np.array(valid_H5['eval_spikes_heldout'])

test_spikes_heldin = np.array(test_H5['eval_spikes_heldin'])

N_tra, T, C = train_spikes_heldin.shape
N_val = eval_spikes_heldin.shape[0]
N_tes = test_spikes_heldin.shape[0]
_, To, Co = train_spikes_heldout_forward.shape

In [6]:
train_fd = np.concatenate([train_spikes_heldin_forward, train_spikes_heldout_forward], axis=-1).astype(np.int64)
train_bd = np.concatenate([train_spikes_heldin, train_spikes_heldout], axis=-1).astype(np.int64)
train = np.concatenate([train_fd, train_bd], axis=1)

## Time Counting

In [7]:
tEnd = time.time()
print ("\n" + "It cost {:.4f} sec" .format(tEnd-tStart))


It cost 1.2952 sec
